In [1]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import spacy
import string
import regex as re
from unidecode import unidecode
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [37]:
reviews = pd.read_csv("McDonald_s_Reviews.csv", encoding='latin1')

In [38]:
reviews

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5 stars


In [39]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Obtenha a data atual
current_date = datetime.now()

# Defina o intervalo de um ano
start_date = current_date
end_date = current_date + timedelta(days=365)

# Crie uma nova coluna de datas aleatórias dentro do intervalo de um ano
reviews['Random_Date'] = [start_date + timedelta(days=random.randint(0, 365)) for _ in range(len(reviews))]

# Exiba o DataFrame com a nova coluna de datas aleatórias
print(reviews)


       reviewer_id  store_name              category  \
0                1  McDonald's  Fast food restaurant   
1                2  McDonald's  Fast food restaurant   
2                3  McDonald's  Fast food restaurant   
3                4  McDonald's  Fast food restaurant   
4                5  McDonald's  Fast food restaurant   
...            ...         ...                   ...   
33391        33392  McDonald's  Fast food restaurant   
33392        33393  McDonald's  Fast food restaurant   
33393        33394  McDonald's  Fast food restaurant   
33394        33395  McDonald's  Fast food restaurant   
33395        33396  McDonald's  Fast food restaurant   

                                           store_address  latitude   \
0      13749 US-183 Hwy, Austin, TX 78750, United States  30.460718   
1      13749 US-183 Hwy, Austin, TX 78750, United States  30.460718   
2      13749 US-183 Hwy, Austin, TX 78750, United States  30.460718   
3      13749 US-183 Hwy, Austin, TX 78750, 

In [40]:
reviews[['endereco', 'estado','cep','pais','aleatorio']] = reviews['store_address'].str.split(', ', expand=True)

In [41]:
reviews

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating,Random_Date,endereco,estado,cep,pais,aleatorio
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,2025-04-07 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,2024-06-24 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star,2025-03-09 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,2024-10-08 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,2024-05-17 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star,2024-10-06 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars,2024-08-02 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars,2024-09-22 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5 stars,2024-05-27 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None


In [42]:
reviews['rating'] = reviews['rating'].str.extract('(\d+)')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Victor\AppData\Local\Temp\ipykernel_5512\2882021106.py:1: SyntaxWarning: invalid escape sequence '\d'
  reviews['rating'] = reviews['rating'].str.extract('(\d+)')


In [43]:
reviews

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating,Random_Date,endereco,estado,cep,pais,aleatorio
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1,2025-04-07 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4,2024-06-24 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1,2025-03-09 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,2024-10-08 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1,2024-05-17 00:22:35.064191,13749 US-183 Hwy,Austin,TX 78750,United States,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1,2024-10-06 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5,2024-08-02 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4,2024-09-22 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5,2024-05-27 00:22:35.064191,3501 Biscayne Blvd,Miami,FL 33137,United States,None


In [46]:
# Supondo que você queira dropar as colunas store_name, category, store_address, rating_count, review_time, endereco, cep e aleatorio
columns_to_drop = ['store_name', 'category', 'store_address', 'rating_count', 'review_time', 'endereco', 'cep', 'aleatorio']

# Dropar as colunas especificadas
reviews = reviews.drop(columns=columns_to_drop)

KeyError: "['store_name', 'category', 'store_address', 'rating_count', 'review_time', 'endereco', 'cep', 'aleatorio'] not found in axis"

In [69]:
df = reviews.copy()

# 5. Data Processing

In [70]:
def to_lower(text):
    if isinstance(text, str):
        return text.lower()
    else:
        return text

In [71]:
df['clean_text'] = df['review'].apply(to_lower)

In [72]:
def remove_punctuation(text):
    if isinstance(text, str):
        punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        punct = punct.replace('"', "")  # Removendo as aspas duplas
        translator = str.maketrans('', '', punct)
        return text.translate(translator)
    else:
        return text

In [73]:
df['clean_text'] = df['clean_text'].apply(remove_punctuation)

In [74]:
def remove_chacacter(text):
    if isinstance(text, str):
        emoji_pattern = re.compile("["
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U000024C2-\U0001F251"  # symbols & pictographs
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)
    else:
        return text

In [75]:
df['clean_text'] = df['clean_text'].apply(remove_chacacter)

In [76]:
def remove_urls(text):
    if isinstance(text, str):
        http = r"http\S+|www\.\S+"
        pattern = re.compile(r"({})".format(http))
        return pattern.sub(r'', text)
    else:
        return text


In [77]:
df['clean_text'] = df['clean_text'].apply(remove_urls)

In [78]:
def remove_numbers(text):
    if isinstance(text, str):
        return re.sub(r'\d+', '', text)
    else:
        return text

In [79]:
df['clean_text'] = df['clean_text'].apply(remove_numbers)

In [80]:
def remove_broke_line(texto):
    if isinstance(texto, str):
        return texto.replace('\n', ' ')
    else:
        return texto

In [81]:
df['clean_text'] = df['clean_text'].apply(remove_broke_line)

In [82]:
df

,reviewer_id,latitude,longitude,review,rating,Random_Date,estado,pais,clean_text
0,1,30.460718,-97.792874,Why does it look like someone spit on my food?...,1,2025-04-07 00:22:35.064191,Austin,United States,why does it look like someone spit on my food ...
1,2,30.460718,-97.792874,It'd McDonalds. It is what it is as far as the...,4,2024-06-24 00:22:35.064191,Austin,United States,itd mcdonalds it is what it is as far as the f...
2,3,30.460718,-97.792874,Made a mobile order got to the speaker and che...,1,2025-03-09 00:22:35.064191,Austin,United States,made a mobile order got to the speaker and che...
3,4,30.460718,-97.792874,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,2024-10-08 00:22:35.064191,Austin,United States,my mc crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿½...
4,5,30.460718,-97.792874,"I repeat my order 3 times in the drive thru, a...",1,2024-05-17 00:22:35.064191,Austin,United States,i repeat my order times in the drive thru and...
...,...,...,...,...,...,...,...,...,...
33391,33392,25.810000,-80.189098,They treated me very badly.,1,2024-10-06 00:22:35.064191,Miami,United States,they treated me very badly
33392,33393,25.810000,-80.189098,The service is very good,5,2024-08-02 00:22:35.064191,Miami,United States,the service is very good
33393,33394,25.810000,-80.189098,To remove hunger is enough,4,2024-09-22 00:22:35.064191,Miami,United States,to remove hunger is enough
33394,33395,25.810000,-80.189098,"It's good, but lately it has become very expen...",5,2024-05-27 00:22:35.064191,Miami,United States,its good but lately it has become very expensive


In [83]:
def strip_text(text):
    if isinstance(text, str):
        return text.strip()
    else:
        return text

In [84]:
df['clean_text'] = df['clean_text'].apply(strip_text)

In [85]:
# df = df.dropna(subset=['clean_text'])

In [86]:
# Supondo que as colunas que você deseja selecionar estejam presentes em df_merged
selected_columns2 = ['clean_text']

# Selecionar apenas as colunas desejadas
df_train = df[selected_columns2]

In [87]:
df_train

,clean_text
0,why does it look like someone spit on my food ...
1,itd mcdonalds it is what it is as far as the f...
2,made a mobile order got to the speaker and che...
3,my mc crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿½...
4,i repeat my order times in the drive thru and...
...,...
33391,they treated me very badly
33392,the service is very good
33393,to remove hunger is enough
33394,its good but lately it has become very expensive


In [88]:
from tensorflow.keras.models import load_model

# Carregar o modelo
model = load_model("kerasmodelLSTM.h5")

In [93]:
# Pré-processamento dos dados de revisão
# 1. Tokenização
tokenizer = Tokenizer(num_words=696000)
tokenizer.fit_on_texts(df_train['clean_text'])
# 2. Convertendo textos em sequências de números
sequences = tokenizer.texts_to_sequences(df_train['clean_text'])
# 3. Preenchimento das sequências
max_length = 100  # Substitua pelo mesmo valor usado durante o treinamento
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Realizar previsões usando o modelo carregado
predictions = model.predict(padded_sequences)

# Decodificar as previsões (por exemplo, encontrar o rótulo com a probabilidade mais alta)
# Suponha que as previsões são codificadas como [0, 1, 2] para [Negativo, Neutro, Positivo]
predicted_labels = [np.argmax(prediction) for prediction in predictions]
# Se você precisar mapear os índices de rótulo de volta para as categorias originais, faça isso aqui

# Adicionar as previsões ao DataFrame de revisão, se necessário
reviews['predicted_label'] = predicted_labels
# Se desejar, você pode salvar o DataFrame resultante em um arquivo CSV

# Exibir o DataFrame com as previsões
print(predicted_labels)

1044/1044 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 1, 0, 2, 1, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 1, 2, 2, 2, 0, 1, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 1, 0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 1, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 0, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 2, 0, 0, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 1, 0, 0, 2, 0, 2, 0, 2, 0, 2, 1, 1, 0, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 1, 2, 2, 2, 0, 1, 1, 2, 1, 1, 2, 2, 2, 0, 1, 2, 1, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1, 0, 2, 0, 1, 0, 0, 1, 1, 2, 1, 0, 0, 2, 1, 0, 0, 1, 0, 1, 1, 1, 2, 2, 

In [109]:
# Renomear as colunas no DataFrame reviews
reviews = reviews.rename(columns={'reviewer_id': 'review_id',
                                  'review': 'review_comment_message',
                                  'rating': 'review_score',
                                  'predicted_label': 'predictions',
                                  'latitude ': 'geolocation_lat',
                                  'longitude': 'geolocation_lng',
                                  'estado': 'geolocation_state',
                                  'pais': 'geolocation_country',
                                  'Random_Date': 'review_creation_date'})

reviews

,review_id,geolocation_lat,geolocation_lng,review_comment_message,review_score,review_creation_date,geolocation_state,geolocation_country,predictions,origin
0,1,30.460718,-97.792874,Why does it look like someone spit on my food?...,1,2025-04-07 00:22:35.064191,Austin,United States,2,tb_mcdonalds
1,2,30.460718,-97.792874,It'd McDonalds. It is what it is as far as the...,4,2024-06-24 00:22:35.064191,Austin,United States,2,tb_subway
2,3,30.460718,-97.792874,Made a mobile order got to the speaker and che...,1,2025-03-09 00:22:35.064191,Austin,United States,2,tb_subway
3,4,30.460718,-97.792874,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,2024-10-08 00:22:35.064191,Austin,United States,2,tb_subway
4,5,30.460718,-97.792874,"I repeat my order 3 times in the drive thru, a...",1,2024-05-17 00:22:35.064191,Austin,United States,2,tb_subway
...,...,...,...,...,...,...,...,...,...,...
33391,33392,25.810000,-80.189098,They treated me very badly.,1,2024-10-06 00:22:35.064191,Miami,United States,0,tb_burgerking
33392,33393,25.810000,-80.189098,The service is very good,5,2024-08-02 00:22:35.064191,Miami,United States,1,tb_burgerking
33393,33394,25.810000,-80.189098,To remove hunger is enough,4,2024-09-22 00:22:35.064191,Miami,United States,1,tb_burgerking
33394,33395,25.810000,-80.189098,"It's good, but lately it has become very expen...",5,2024-05-27 00:22:35.064191,Miami,United States,2,tb_mcdonalds


In [100]:
reviews['origin'] = np.random.choice(['tb_mcdonalds', 'tb_burgerking', 'tb_subway'], size=len(reviews))

In [110]:
reviews

,review_id,geolocation_lat,geolocation_lng,review_comment_message,review_score,review_creation_date,geolocation_state,geolocation_country,predictions,origin
0,1,30.460718,-97.792874,Why does it look like someone spit on my food?...,1,2025-04-07 00:22:35.064191,Austin,United States,2,tb_mcdonalds
1,2,30.460718,-97.792874,It'd McDonalds. It is what it is as far as the...,4,2024-06-24 00:22:35.064191,Austin,United States,2,tb_subway
2,3,30.460718,-97.792874,Made a mobile order got to the speaker and che...,1,2025-03-09 00:22:35.064191,Austin,United States,2,tb_subway
3,4,30.460718,-97.792874,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,2024-10-08 00:22:35.064191,Austin,United States,2,tb_subway
4,5,30.460718,-97.792874,"I repeat my order 3 times in the drive thru, a...",1,2024-05-17 00:22:35.064191,Austin,United States,2,tb_subway
...,...,...,...,...,...,...,...,...,...,...
33391,33392,25.810000,-80.189098,They treated me very badly.,1,2024-10-06 00:22:35.064191,Miami,United States,0,tb_burgerking
33392,33393,25.810000,-80.189098,The service is very good,5,2024-08-02 00:22:35.064191,Miami,United States,1,tb_burgerking
33393,33394,25.810000,-80.189098,To remove hunger is enough,4,2024-09-22 00:22:35.064191,Miami,United States,1,tb_burgerking
33394,33395,25.810000,-80.189098,"It's good, but lately it has become very expen...",5,2024-05-27 00:22:35.064191,Miami,United States,2,tb_mcdonalds


In [111]:
reviews['geolocation_point'] = reviews['geolocation_lat'].astype(str) + ' ' + reviews['geolocation_lng'].astype(str)

In [112]:
reviews

,review_id,geolocation_lat,geolocation_lng,review_comment_message,review_score,review_creation_date,geolocation_state,geolocation_country,predictions,origin,geolocation_point
0,1,30.460718,-97.792874,Why does it look like someone spit on my food?...,1,2025-04-07 00:22:35.064191,Austin,United States,2,tb_mcdonalds,30.4607176 -97.7928744
1,2,30.460718,-97.792874,It'd McDonalds. It is what it is as far as the...,4,2024-06-24 00:22:35.064191,Austin,United States,2,tb_subway,30.4607176 -97.7928744
2,3,30.460718,-97.792874,Made a mobile order got to the speaker and che...,1,2025-03-09 00:22:35.064191,Austin,United States,2,tb_subway,30.4607176 -97.7928744
3,4,30.460718,-97.792874,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,2024-10-08 00:22:35.064191,Austin,United States,2,tb_subway,30.4607176 -97.7928744
4,5,30.460718,-97.792874,"I repeat my order 3 times in the drive thru, a...",1,2024-05-17 00:22:35.064191,Austin,United States,2,tb_subway,30.4607176 -97.7928744
...,...,...,...,...,...,...,...,...,...,...,...
33391,33392,25.810000,-80.189098,They treated me very badly.,1,2024-10-06 00:22:35.064191,Miami,United States,0,tb_burgerking,25.8099996 -80.1890981
33392,33393,25.810000,-80.189098,The service is very good,5,2024-08-02 00:22:35.064191,Miami,United States,1,tb_burgerking,25.8099996 -80.1890981
33393,33394,25.810000,-80.189098,To remove hunger is enough,4,2024-09-22 00:22:35.064191,Miami,United States,1,tb_burgerking,25.8099996 -80.1890981
33394,33395,25.810000,-80.189098,"It's good, but lately it has become very expen...",5,2024-05-27 00:22:35.064191,Miami,United States,2,tb_mcdonalds,25.8099996 -80.1890981


In [113]:
reviews.to_csv('deploycsv.csv', sep=";")

In [106]:
import psycopg2
from sqlalchemy import create_engine

## CONFIG - POSTGRES ##
db_config = {
    'user': 'postgres',
    'password': 'admin',
    'host': 'localhost',
    'port': '5432',
    'database': 'fluffyapi'
}
conn = psycopg2.connect(**db_config)
engine = create_engine('postgresql+psycopg2://' + db_config['user'] + ':' + db_config['password'] + '@' + db_config['host'] + ':' + db_config['port'] + '/' + db_config['database'])

In [116]:
# Substitua 'nome_da_tabela' pelo nome da tabela em seu banco de dados
nome_da_tabela = 'review'

# Inserir o DataFrame reviews no banco de dados PostgreSQL
reviews.to_sql(nome_da_tabela, engine, if_exists='append', index=False)

print("Dados inseridos com sucesso na tabela", nome_da_tabela)

Dados inseridos com sucesso na tabela review


In [127]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from unicodedata import normalize
import string

# Carregar lista de stopwords em inglês
stop_words = set(stopwords.words('english'))

# Função para remover acentuações e palavras pontuadas
def preprocess_text(text):
    # Remover acentuações
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    # Remover palavras pontuadas e stopwords
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation and token.strip()]
    return ' '.join(tokens)

# Função para contar a frequência das palavras e aplicar o pré-processamento
def count_words(text):
    tokens = word_tokenize(text.lower())
    tokens = [preprocess_text(token) for token in tokens]
    return Counter(tokens)

# Filtrar o DataFrame para cada categoria de 'predictions'
predictions_0 = reviews[reviews['predictions'] == 0]['review_comment_message']
predictions_1 = reviews[reviews['predictions'] == 1]['review_comment_message']
predictions_2 = reviews[reviews['predictions'] == 2]['review_comment_message']

# Contagem de palavras e aplicação do pré-processamento para cada categoria
word_counts_0 = predictions_0.apply(count_words).sum()
word_counts_1 = predictions_1.apply(count_words).sum()
word_counts_2 = predictions_2.apply(count_words).sum()

# Selecionar as top 60 palavras mais citadas em cada categoria
top_words_0 = word_counts_0.most_common(60)
top_words_1 = word_counts_1.most_common(60)
top_words_2 = word_counts_2.most_common(60)

# Juntar os resultados em um único DataFrame
top_words_df = pd.DataFrame({
    'word': [word[0] for word in top_words_0 + top_words_1 + top_words_2],
    'counts': [word[1] for word in top_words_0 + top_words_1 + top_words_2],
    'sentiment': ['0'] * 60 + ['1'] * 60 + ['2'] * 60
})

# Exibir o DataFrame com as top 60 palavras mais citadas em cada categoria
print(top_words_df)


        word  counts sentiment
0             133827         0
1         's    2796         0
2       food    2612         0
3      order    2106         0
4    service    1883         0
..       ...     ...       ...
175     took     601         2
176      bad     592         2
177     give     579         2
178       ..     568         2
179   orders     567         2

[180 rows x 3 columns]


In [128]:
# Substitua 'nome_da_tabela' pelo nome da tabela em seu banco de dados
nome_da_tabela = 'palavra'

# Inserir o DataFrame reviews no banco de dados PostgreSQL
top_words_df.to_sql(nome_da_tabela, engine, if_exists='append', index=False)

print("Dados inseridos com sucesso na tabela", nome_da_tabela)

Dados inseridos com sucesso na tabela palavra
